# Data Ingestion

In [1]:
print("###### Starting data Engineering ###### ")

###### Starting data Engineering ###### 


In [2]:
import pandas as pd
import pandas_profiling

In [3]:
books = pd.read_csv("book-recommend-data/Books.csv", encoding= "ISO-8859-1")
books =  books.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
books.head()
#books.shape[0]

,bookISBN,bookName,author,yearOfPublication,publisher,urlId
0,1551669382,Enchanted Afternoon,Susan Wiggs,2002,Mira,120656.0
1,0743202961,Every Breath You Take : A True Story of Obsess...,Ann Rule,2001,Free Press,116803.0
2,067151699X,The HIDDEN LIFE OF DOGS,Elizabeth Marshall Thomas,1995,Pocket,1805.0
3,0804108528,Inadmissible Evidence,Philip Friedman,1993,Ivy Books,139375.0
4,0061094145,"The Fifth Dominion (Imajica, Book 1)",Clive Barker,1995,HarperTorch,141585.0


In [4]:
userEvents = pd.read_csv("book-recommend-data/UserEvents.csv", encoding= "ISO-8859-1")
userEvents = userEvents.drop(['Unnamed: 0'], axis=1)
userEvents.columns = ['user', 'bookISBN','impression']
userEvents.head()
#userEvents.shape[0]

,user,bookISBN,impression
0,276747,0060517794,checkout
1,276747,0671537458,checkout
2,276747,0679776818,add to cart
3,276822,0060096195,checkout
4,276822,0375821813,checkout


In [5]:
users =  pd.read_csv("book-recommend-data/Users.csv", encoding= "ISO-8859-1")
users = users.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
users.head()
#users.shape[0]

,user,location,age
0,43157.0,"windsor, ontario, canada",22.0
1,99032.0,"white plains, new york, usa",25.0
2,42883.0,"the woodlands, texas, usa",51.0
3,183800.0,"quebec, quebec, canada",29.0
4,188796.0,"toronto, ontario, canada",NaN


In [6]:
df = pd.merge(userEvents,users, on='user', how='left')

In [7]:
df.head()
#df.shape[0]

,user,bookISBN,impression,location,age
0,276747,0060517794,checkout,NaN,NaN
1,276747,0671537458,checkout,NaN,NaN
2,276747,0679776818,add to cart,NaN,NaN
3,276822,0060096195,checkout,NaN,NaN
4,276822,0375821813,checkout,NaN,NaN


In [8]:
df = pd.merge(df,books, on='bookISBN', how='left')

# Data profiling

In [9]:
pandas_profiling.ProfileReport(df)

Number of variables,10
Number of observations,141081
Total Missing (%),55.5%
Total size in memory,11.8 MiB
Average record size in memory,88.0 B
Numeric,4
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# Data Adjusts

In [10]:
# Age 
df.age[df.age > df.age.quantile(.95)] = df.age.quantile(.95) # Handling top outlier
df.age[df.age < df.age.quantile(.1)] = df.age.quantile(.1)   # Handling bottom outlier
df.age = df.age.fillna(df.age.median()) # Handling NAN / Initial approach medium -> next step create a submodel to fill missing information

C:\Users\Rodrigo\Anaconda2\envs\bonsai\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Rodrigo\Anaconda2\envs\bonsai\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [11]:
# author
df.author = df.author.fillna(-1) #df[df.author.notnull()] # Handling NAN / Initial approach DROP due to 3% missing values

In [12]:
# location
df.location = df.location.fillna(-1) # .dropna(inplace = True) # Handling NAN / Initial approach DROP due to 1.6% missing values

In [13]:
# Book name
df.bookName = df.bookName.fillna(-1)  # replace by API

In [14]:
# Publisher
df.publisher = df.publisher.fillna(-1)

In [15]:
# Year of Publication
df.yearOfPublication = df.yearOfPublication.fillna(-1)
df.yearOfPublication[df.yearOfPublication == 0 ] = -1

C:\Users\Rodrigo\Anaconda2\envs\bonsai\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [16]:
# urlId
df = df.drop(['urlId'],axis = 1) # Removig

In [17]:
df.shape[0]

141081

In [18]:
df.head()

,user,bookISBN,impression,location,age,bookName,author,yearOfPublication,publisher
0,276747,0060517794,checkout,-1,35.0,-1,-1,-1.0,-1
1,276747,0671537458,checkout,-1,35.0,-1,-1,-1.0,-1
2,276747,0679776818,add to cart,-1,35.0,-1,-1,-1.0,-1
3,276822,0060096195,checkout,-1,35.0,-1,-1,-1.0,-1
4,276822,0375821813,checkout,-1,35.0,-1,-1,-1.0,-1


In [19]:
df.to_csv('book-recommend-data/dataframe.csv',sep=',',index = False)

In [20]:
print("###### Finishing data Engineering ###### ")

###### Finishing data Engineering ###### 
